In [1]:
# generate the symmetry-corrected indices for move-based convolution
import sys
import numpy as np
sys.path.append('..')

In [2]:
# test the coefficient generation logic the naive way
cell = 13

In [3]:
# display-only code, to visually check the coeffs are in the correct locations on the board
from neural.neural_ import to_pair, generate_all_moves_by_index, move_convolution_indices
from constants import BOARD_SIZE, BOARD_WIDTH, NUM_BIASES
from math import floor

all_inds, num_coeffs = move_convolution_indices()


num_coeffs -= NUM_BIASES # the first 10 in the above function are biases, don't need them
num_fields = BOARD_SIZE

cell = cell+1
tmp = all_inds[cell]
a = np.zeros([BOARD_WIDTH,BOARD_WIDTH])
for (ind, coeff) in tmp[1:]:
    pair = to_pair(ind)
    a[pair[0],pair[1]] = coeff - NUM_BIASES + 1

print(to_pair(cell))
print(a)

(2, 3)
[[ 0.  0.  6.  0.]
 [ 0.  5.  0.  0.]
 [ 0.  0.  0.  2.]
 [ 0.  7.  0.  0.]]


In [4]:
# try calling conv_stack directly from Tensorflow
import numpy as np
import tensorflow as tf

tf.reset_default_graph()

from neural.tensorflow_utils import conv_stack

with tf.Session() as sess:
    in_fields_np = np.ones([2,num_fields])
    in_fields_np[0,3] = 0
    in_fields_np[0,5] = 0
    my_pos = np.array([12, 12])
    other_pos =  np.array([5,5])
    inputs_np = np.concatenate([in_fields_np, my_pos[:,None], other_pos[:,None]],
                              1)
    #print(inputs_np.shape)
    inputs =tf.constant(inputs_np)# tf.placeholder(shape =[None, num_fields+2], dtype = tf.float32) #
    #print(sess.run(inputs))
    out = conv_stack(inputs, 5,sess)
    
    dummy = np.array([50,50])[:,None]
    #print(sess.run(get_random_index(inputs, tf.constant(dummy))))
    
    sess.run(tf.global_variables_initializer())
    stack_result = sess.run(out)#, feed_dict={inputs:inputs_np})
    print(stack_result)

[2 3 2]
[[[ 0.16027431  0.        ]
  [-0.29731864 -0.        ]
  [-0.46966732 -0.        ]]

 [[ 0.65186518  1.36244631]
  [-0.02570902 -0.05224879]
  [ 0.03826804  1.73317242]]]


In [5]:
# load game simulation data
import glob
import sys
import pickle
from neural.data_utils import load_simulation_data

fn = '../data/states.pickle'
if True:
# try:
#     with open(fn, 'rb') as f:
#         states = pickle.load(f)
# except:
    files = glob.glob('../data/4x4tiny.pickle')
    #files = glob.glob('../data/epsgreedy/*')
    #files = glob.glob('../data/ID_x2_1000ms/result_ID*.pickle')
    #print(files)
    depths =load_simulation_data(files)
    keys = list(depths.keys())
    #print(keys)
    games = depths[keys[0]]
    #print(games[0])
    states = [state for game in games for state in game if 'score' in state] 
    print(len(states))
    with open(fn, 'wb') as f:
        pickle.dump(states,f)

1271


In [6]:
from neural.data_utils import prepare_data_for_model
board_full, player_pos, y = prepare_data_for_model(states,'simple_score')
print(board_full.shape, player_pos.shape, y.shape)
y[y ==float('-inf')] = 0
y[y ==float('inf')] = 0

(1271, 16, 1) (1271, 16, 2) (1271, 1)


In [7]:
# fit the naive score as a first test of our network
from neural.keras_utils import deep_model_fun
model = deep_model_fun(num_features = 8, num_res_modules = 4, drop_rate = 0.01, activation = 'linear')
model.summary()
model.compile(optimizer = 'adam',  loss='mean_squared_error')

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 16, 1)         0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, 16, 2)         0                                            
____________________________________________________________________________________________________
concatenate_1 (Concatenate)      (None, 16, 3)         0           input_2[0][0]                    
                                                                   input_1[0][0]                    
____________________________________________________________________________________________________
conv_by_move_layer_1 (ConvByMove (None, 16, 8)         240         concatenate_1[0][0]     

In [8]:
model.fit(x = [player_pos, board_full],y = y, batch_size = 16, epochs=30, verbose =1)

Epoch 1/30
1271/1271 [==============================] - 6s - loss: 9.9684      
Epoch 2/30
1271/1271 [==============================] - 3s - loss: 4.3643     
Epoch 3/30
1271/1271 [==============================] - 3s - loss: 3.4885     
Epoch 4/30
1271/1271 [==============================] - 3s - loss: 3.0461     
Epoch 5/30
1271/1271 [==============================] - 3s - loss: 2.8112     
Epoch 6/30
1271/1271 [==============================] - 3s - loss: 2.2658     
Epoch 7/30
1271/1271 [==============================] - 3s - loss: 2.0466     
Epoch 8/30
1271/1271 [==============================] - 3s - loss: 1.6279     
Epoch 9/30
1271/1271 [==============================] - 3s - loss: 1.5493     
Epoch 10/30
1271/1271 [==============================] - 3s - loss: 1.5620     
Epoch 11/30
1271/1271 [==============================] - 3s - loss: 1.1928     
Epoch 12/30
1271/1271 [==============================] - 3s - loss: 1.1318     
Epoch 13/30
1271/1271 [=========================

In [9]:
states[0]

{'G': 0.96059601,
 'active_player': None,
 'allscores': None,
 'depth': 11,
 'game': array([ 1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.]),
 'game_': None,
 'move': (0, 1),
 'pos': (2, 5),
 'score': inf,
 'simple_score': -8.0,
 'winner': 1.0}

In [10]:
test = [s for s,state in enumerate(states) if 'score' not in state ]
print(test)

[]


In [16]:
# Now let's get all those games where tree search actually completed
import numpy as np
from sklearn.preprocessing import OneHotEncoder
test = ['score' not in state for state in states]
complete_states = [state for state in states if state['score'] == float('inf') or state['score'] == float('-inf')]
print(len(complete_states))
board_full_c, player_pos_c, y_c = prepare_data_for_model(complete_states,'score')
y_c[y_c==float('inf')] = 1
y_c[y_c==float('-inf')] = 0

1271


In [17]:
print(set(list(np.reshape(y_c,[-1]))))

{0.0, 1.0}


In [23]:
from neural.keras_utils import deep_model_fun
# width AND depth matter! (32, 8 seem about optimal on this dataset)
deep_model = deep_model_fun(num_features =8, num_res_modules = 8, drop_rate = 0.05, activation = 'sigmoid')
deep_model.summary()
#deep_model.compile(optimizer = 'adam',  loss='binary_crossentropy', metrics =['acc'])deep_model.compile(optimizer = 'adam',  loss='binary_crossentropy', metrics =['acc'])
deep_model.compile(optimizer = 'adam',  loss='mean_squared_error', metrics =['acc'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_10 (InputLayer)            (None, 16, 1)         0                                            
____________________________________________________________________________________________________
input_9 (InputLayer)             (None, 16, 2)         0                                            
____________________________________________________________________________________________________
concatenate_9 (Concatenate)      (None, 16, 3)         0           input_10[0][0]                   
                                                                   input_9[0][0]                    
____________________________________________________________________________________________________
conv_by_move_layer_61 (ConvByMov (None, 16, 8)         240         concatenate_9[0][0]     

In [ ]:
deep_model.fit([player_pos_c, board_full_c], y_c, batch_size = 16, epochs=30, verbose =1, validation_split = 0.2, shuffle = True)

Train on 1016 samples, validate on 255 samples
Epoch 1/30
1016/1016 [==============================] - 14s - loss: 0.3331 - acc: 0.5719 - val_loss: 0.2948 - val_acc: 0.6863
Epoch 2/30
1016/1016 [==============================] - 5s - loss: 0.2537 - acc: 0.6732 - val_loss: 0.2612 - val_acc: 0.7176
Epoch 3/30
1016/1016 [==============================] - 5s - loss: 0.2303 - acc: 0.6959 - val_loss: 0.2434 - val_acc: 0.7216
Epoch 4/30
1016/1016 [==============================] - 5s - loss: 0.2221 - acc: 0.6978 - val_loss: 0.2355 - val_acc: 0.7216
Epoch 5/30
1016/1016 [==============================] - 5s - loss: 0.2154 - acc: 0.7057 - val_loss: 0.2295 - val_acc: 0.7216
Epoch 6/30
1016/1016 [==============================] - 5s - loss: 0.2016 - acc: 0.7352 - val_loss: 0.2292 - val_acc: 0.7216
Epoch 7/30
1016/1016 [==============================] - 5s - loss: 0.1947 - acc: 0.7402 - val_loss: 0.2190 - val_acc: 0.7216
Epoch 8/30
1016/1016 [==============================] - 5s - loss: 0.1866 - a

In [15]:
from collections import namedtuple
from copy import copy
from neural.neural_agent import apply_move, get_best_move_from_model

board = np.ones(49)
#print(list(board))
board.sum()
my_pos = None
other_pos = None
game = {'pos': np.array([my_pos, other_pos]), 'game': board}
game1 = apply_move(game, 0)
game2 = apply_move(game1, 1)
game3 = apply_move(game2, 15)
# board, pos, _ = possible_moves_for_model(game3)
# print(board.shape, pos.shape)
get_best_move_from_model(game3, model)


ValueError: Illegal move!

In [ ]:
%load_ext autoreload
%autoreload 2
from neural.neural_agent import NeuralAgent
my_agent = NeuralAgent(deep_model)
my_agent.get_move(game3)

In [ ]:
from tournament import tournament, Agent, RandomPlayer
from neural.neural_agent import NeuralAgent

my_agent = NeuralAgent(deep_model)
tournament(num_matches=20, time_limit=float('inf'), 
           test_agents=[Agent(my_agent,"Neural Agent")])
a=1

In [ ]:
# sort all games by number of moves. 
states_by_num_moves = [[] for _ in range(BOARD_SIZE)]

for state in states:
    moves_made = BOARD_SIZE - state['game'].sum()
    states_by_num_moves[int(moves_made)].append(state)
    
for n in range(BOARD_SIZE):
    print(n,len(states_by_num_moves[n]))

In [ ]:
# Iteratively populate all non-+-inf values in layer n from evaluating model in layer n+1, then include these into the fitting set
# after each pass, refresh the values for earlier layers
def recursively_fill_scores(states, model = deep_model):
    print(len(states))
    scores = np.zeros([len(states)])
    for n,state in enumerate(states):
        if state['score']  == float('inf'):
            scores[n] = 1
        elif state['score'] == float('-inf'):
            scores[n] = 0
        else:
            _ , scores[n] = get_best_move_from_model(state, model)
        if n%1000 == 0:
            print(n)
    return scores

prepared_data = [None for _ in range(49)]

for n in range(18,BOARD_SIZE):
    if len(states_by_num_moves[n]):
        board, pos, _  = prepare_data_for_model( states_by_num_moves[n], None) # board, player_pos, score
        scores = recursively_fill_scores( states_by_num_moves[n])
        prepared_data[n] = (pos, board, scores)
        print(len(scores),len(set(list(scores))))
# TODO: is my position always first in those dumps???
    

            


            
